# MTMount accelerometers
Querying MTMount accelerometers \
Craig Lage - 22-Mar-23

In [ ]:
import sys, time, os, asyncio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst_efd_client.efd_helper import EfdClient, merge_packed_time_series

In [ ]:
# Get EFD client
client = EfdClient('summit_efd')

In [ ]:
# Times to look at the data
start = Time("2023-03-22 08:00:00Z", scale='utc')
end = Time("2023-03-22 08:10:00Z", scale='utc')

In [ ]:
baseFields = ['accelerationX', 'accelerationY', 'accelerationZ']
sensorNames = ['SST top end ring +x -y', 'SST top end ring -x -y', 'SST spider spindle', 'SST M2 surrogate'] 

##  Get all of the data for the selected times

In [ ]:
packed_dataframe = await client.select_time_series("lsst.sal.ESS.accelerometer", ["*"], start, end)

## Now unpack it and plot it by sensor

In [ ]:
fig, axs = plt.subplots(2,2, figsize = (8,8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
for i, sensorName in enumerate(sensorNames):
    sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName==sensorName]
    plotx = i%2
    ploty = int(i/2)
    ax = axs[plotx][ploty]
    ax.set_title(sensorName)
    ax.set_ylabel("Accel(m/s^2)")
    for baseField in baseFields:
        df = merge_packed_time_series(sub_dataframe, baseField, stride=1,
                             ref_timestamp_col="timestamp", fmt='unix_tai',
                             scale='tai')
        df[baseField].plot(ax=ax, label=list(baseField)[-1])
    ax.legend()
plt.savefig("/home/craiglagegit/DATA/MT_accels/MT_Accel_22Mar23.png")

## Testing notebook_vandv version

In [ ]:
def unpack_tma_accel(df, axis, time_format="seconds"):
    """Unpack TMA accelerometer data from given dataframe.

    Parameters
    ----------
    df: pd.DataFrame
        The dataframe of values.
    axis: str
        Must be "X", "Y", or "Z".
    time_format: str
        If "seconds", returns relative time in seconds.
        If "stamps", returns time stamps.

    Returns
    -------
    np.ndarray
        Time of acceleration data.
    np.ndarray
        Accelerations in m/s^2.
    """
    # validate parameters
    if axis not in ["X", "Y", "Z"]:
        raise ValueError(f"axis {axis} invalid; must be 'X', 'Y', or 'Z'.")
    if time_format not in ["seconds", "stamps"]:
        raise ValueError(
            f"time_format {time_format} invalid; must be 'seconds' or 'stamps'."
        )

    # now, let's make sure the timestamps are in order
    df = df.sort_index()

    # pull out the initial time stamp and the intervals
    stamp0 = df.index[0]
    intervals = df.interval.to_numpy()

    # select acceleration columns for the specified axis
    df = df[[col for col in df.columns if axis in col]]

    # rename columns with integer time step
    df = df.rename(columns=lambda col: int(col.split(axis)[1]))

    # put the columns in order
    df = df.sort_index(axis=1)

    # convert index to dt in seconds
    df.index = (df.index - df.index[0]).total_seconds()

    # pull out the times (again, in seconds)
    row_times = df.index.to_numpy()
    column_times = intervals[:, None] * df.columns.to_numpy()[None, :]
    times = row_times[:, None] + column_times

    # finally, extract the times and accelerometer data
    t = times.flatten()
    accel = df.to_numpy().flatten()

    # convert to time stamps?
    if time_format == "stamps":
        t = stamp0 + pd.to_timedelta(t, "s")

    return t, accel

In [ ]:
axes = ['X', 'Y', 'Z']
sensorNames = ['SST top end ring +x -y', 'SST top end ring -x -y', 'SST spider spindle', 'SST M2 surrogate'] 
fig, axs = plt.subplots(2,2, figsize = (8,8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
for i, sensorName in enumerate(sensorNames):
    sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName==sensorName]
    plotx = i%2
    ploty = int(i/2)
    ax = axs[plotx][ploty]
    ax.set_title(sensorName)
    ax.set_ylabel("Accel(m/s^2)")
    for axis in axes:
        t, accel = unpack_tma_accel(sub_dataframe, axis=axis) 
        ax.plot(t, accel, label=axis)
        ax.legend()
